## Import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Read csv

In [2]:
file = "Pure/pure_pids.csv"
df = pd.read_csv(file, usecols=['objid', 'bands', 'pid1', 'pid2'], dtype=object)
total = len(df)
print("Total objects = {}".format(total))

Total objects = 973


## Band-Count and Object-Bands Dicts

In [3]:
band_stat = {b:[] for b in "ugri"}
objid_stat = {}

for i in range(total) :
    objid = df.loc[i, 'objid']
    bands = df.loc[i, 'bands']
    for b in bands :
        band_stat[b].append(df.loc[i, 'objid'])
        
    objid_stat[objid] = bands

## Band Correlation

In [4]:
corrs = {}
for b1 in "ugri" :
    corr = {b:0 for b in "ugri" if b != b1}

    other_count = []
    single_band = 0
    for objid in band_stat[b1] :
        other = 0
        for b2 in objid_stat[objid] :
            if b1 != b2 :
                other += 1
                corr[b2] += 1

        if not other :
            single_band += 1
        else :
            other_count.append(other)

    uq, cnts = np.unique(np.array(other_count), return_counts=True)
    other_count = {uq[i]:cnts[i] for i in range(len(uq))}
    corrs[b1] = (corr, single_band, other_count)

# Overall Band Statistics

In [24]:
max_corr = max([
    max([u for u,_ in corrs[b][2].items()])
])
print ("Maximum number of bands in which an object was detected simultaneously = {}".format(max_corr))

max_band, max_count = '', 0
for b in "ugri" :
    if len(band_stat[b]) > max_count :
        max_count = len(band_stat[b])
        max_band = b
print ("Band in which maximum detections occured is '{}' with {} detections".format(max_band, max_count))

Maximum number of bands in which an object was detected simultaneously = 2
Band in which maximum detections occured is 'i' with 597 detections


## Individual Band Statitics

In [26]:
b = 'u'
print ("Total detections in band = {}".format(len(band_stat[b])))
print ("Only detections in band = {}".format(corrs[b][1]))
print ("Correlated detections in band = {}".format(corrs[b][0]))
print ("Number of correlations = {}".format(corrs[b][2]))

Total detections in band = 138
Only detections in band = 133
Correlated detections in band = {'g': 2, 'r': 1, 'i': 3}
Number of correlations = {1: 4, 2: 1}
